# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.16,67,100,12.53,TF,1686459062
1,1,margaret river,-33.9500,115.0667,13.51,62,97,8.98,AU,1686459062
2,2,pacasmayo,-7.4006,-79.5714,21.72,83,14,5.84,PE,1686459062
3,3,papatowai,-46.5619,169.4708,9.02,83,60,1.58,NZ,1686459440
4,4,veinticinco de mayo,-35.4323,-60.1727,4.97,56,0,5.51,AR,1686459440


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [50]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    #"Country",
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    hover_cols = "Country"
)

# Display the map plot
hvplot.show(map_plot_1)

Launching server at http://localhost:53010


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 3: Create a new DataFrame called `hotel_df`.

In [ ]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.